# Packages

In [1]:
import torch
import torchvision
from PIL import Image
from torchvision import transforms, models
from torch.utils import data
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import numpy as np
import pandas as pd
from skimage import io
from PIL import Image
import time
import copy

from tqdm import tqdm

import os
import shutil
import re

base_dir = '/home/nvme/data/openimg'

# Create training folder
files = os.listdir(base_dir + '/train')
print(files)

print("test")

['Caterpillar', 'Car', 'Truck', 'Segway', 'Limousine', 'Motorcycle', 'Tank', 'Barge', 'Helicopter', 'Bicycle', 'Bus', 'Snowmobile', 'Boat', 'Cart', 'Taxi', 'Van', 'Ambulance']


# Pathes and class encoding

In [2]:
dataset = torchvision.datasets.ImageFolder(base_dir + '/train')
pathes = dataset.samples[:len(dataset)]
for i in range(len(pathes)):
    pathes[i] = pathes[i][0]
labels = dataset.targets
pathes[10984], labels[10984]

('/home/nvme/data/openimg/train/Bus/000343_12.jpg', 4)

In [3]:
path = base_dir + '/train/'
data = []
for category in sorted(os.listdir(path)):
    for file in sorted(os.listdir(os.path.join(path, category))):
        data.append((category, os.path.join(path, category,  file)))

df = pd.DataFrame(data, columns=['class', 'file_path'])
counts = df['class'].value_counts().to_dict()
df_counts = dataset.class_to_idx.copy()
for key in df_counts:
    df_counts[key] = counts[key]
df_counts

{'Ambulance': 132,
 'Barge': 202,
 'Bicycle': 1618,
 'Boat': 8695,
 'Bus': 2133,
 'Car': 6782,
 'Cart': 51,
 'Caterpillar': 331,
 'Helicopter': 668,
 'Limousine': 74,
 'Motorcycle': 2986,
 'Segway': 153,
 'Snowmobile': 123,
 'Tank': 206,
 'Taxi': 748,
 'Truck': 2033,
 'Van': 1111}

# Custom dataset loader 

In [4]:
import torch
from torch.utils import data

class Dataset(data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, list_IDs, pathes, labels, phase, transforms=None):
        'Initialization'
        self.pathes = pathes
        self.labels = labels
        self.list_IDs = list_IDs
        self.transforms = transforms
        self.phase = phase
    
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)
    
    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]
        
        # Load data and get label
        
        img = Image.open(self.pathes[ID])
        if self.transforms is not None:
            X = self.transforms(img)
            
        if self.phase in ['train', 'val']:
            y = self.labels[ID]
            return X, y
        else:
            return X, None

# Train / val random split 

In [5]:
def get_subset(indices, start, end):
    return indices[start : start + end]


TRAIN_PCT, VALIDATION_PCT = 0.75, 0.25  # rest will go for test
train_count = int(len(dataset) * TRAIN_PCT)
validation_count = int(len(dataset) * TRAIN_PCT)

indices = torch.randperm(len(dataset))

train_indices = get_subset(indices, 0, train_count)
validation_indices = get_subset(indices, train_count, validation_count)

# Custom Sampler

In [6]:
class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    """Samples elements randomly from a given list of indices for imbalanced dataset
    Arguments:
        indices (list, optional): a list of indices
        num_samples (int, optional): number of samples to draw
    """

    def __init__(self, dataset, indices=None, num_samples=None):
                
        # if indices is not provided, 
        # all elements in the dataset will be considered
        self.indices = list(range(len(dataset))) \
            if indices is None else indices
            
        # if num_samples is not provided, 
        # draw `len(indices)` samples in each iteration
        self.num_samples = len(self.indices) \
            if num_samples is None else num_samples
            
        # distribution of classes in the dataset 
        label_to_count = {}
        for idx in self.indices:
            label = self._get_label(dataset, idx)
            if label in label_to_count:
                label_to_count[label] += 1
            else:
                label_to_count[label] = 1
                
        # weight for each sample
        weights = [1.0 / label_to_count[self._get_label(dataset, idx)]
                   for idx in self.indices]
        self.weights = torch.DoubleTensor(weights)

    def _get_label(self, dataset, idx):
        i = dataset.list_IDs[idx]
        return dataset.labels[i]
                
    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(
            self.weights, self.num_samples, replacement=True))

    def __len__(self):
        return self.num_samples

# Parameters and datasets

In [7]:
train_transforms = transforms.Compose([           
    transforms.Resize(256),                    
    transforms.CenterCrop(224),                
    transforms.ToTensor(),                     
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],                
        std=[0.229, 0.224, 0.225]
    )
])

val_transforms = transforms.Compose([          
    transforms.Resize(256),                    
    transforms.CenterCrop(224),               
    transforms.ToTensor(),                     
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],               
        std=[0.229, 0.224, 0.225]
    )
])

# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#cudnn.benchmark = True # not working

# Parameters
params = {'batch_size': 64,
          'num_workers': 6}
max_epochs = 100

# Datasets
partition = {'train': train_indices.tolist(), 'validation': validation_indices.tolist()}
labels = {}
for i in indices:
    labels[int(i)] = dataset.samples[indices[i]][1]

# Generators
training_set = Dataset(partition['train'], pathes, labels, 'train', train_transforms)
training_generator = data.DataLoader(training_set, **params
                                     , sampler = ImbalancedDatasetSampler(training_set, num_samples = 17000)
                                    )

validation_set = Dataset(partition['validation'], pathes, labels, 'val', val_transforms)
validation_generator = data.DataLoader(validation_set, **params
                                      , sampler = ImbalancedDatasetSampler(validation_set, num_samples = 5661)
                                      )

dataloaders = {'train': training_generator, 'val' : validation_generator}
dataset_sizes = {'train': len(training_set.list_IDs), 'val': len(validation_set.list_IDs)}

# ResNet

In [8]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()


#resnet = models.resnet(pretrained=True)
#num_ftrs = resnet.fc.in_features
#resnet.fc = nn.Linear(num_ftrs, 17)

vgg = models.vgg11_bn(pretrained=False)
#set_parameter_requires_grad(vgg, feature_extract)
num_ftrs = vgg.classifier[6].in_features
vgg.classifier[6] = nn.Linear(num_ftrs,17)
input_size = 224

for name, child in vgg.named_children():
    print(name)

/home/esa/miniconda3/envs/openimg/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/esa/miniconda3/envs/openimg/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/esa/miniconda3/envs/openimg/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/esa/miniconda3/envs/op

features
avgpool
classifier


In [9]:
vgg = vgg.to(device)
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(vgg.parameters(), lr=3e-4)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [10]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    iter_num_train = 0
    iter_num_val = 0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            current_loss = 0.0
            current_corrects = 0

            # Here's where the training happens
            print('Iterating through data...')

            for inputs, labels in tqdm(dataloaders[phase], desc=f'{phase} epoch ({epoch})'):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # We need to zero the gradients, don't forget it
                optimizer.zero_grad()

                # Time to carry out the forward training poss
                # We only need to log the loss stats if we are in training phase
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # We want variables to hold the loss statistics
                current_loss += loss.item() * inputs.size(0)
                current_corrects += torch.sum(preds == labels.data)
                
                #writer.add_scalar(f'Loss/{phase}', loss.item()/params['batch_size'], iter_num)
                #writer.add_scalar(f'Accuracy/{phase}', torch.sum(preds == labels.data)/params['batch_size'], iter_num)
                
                if phase == "train":
                    writer.add_scalar(f'Loss/{phase}', loss.item()/params['batch_size'], iter_num_train)
                    writer.add_scalar(f'Accuracy/{phase}', torch.sum(preds == labels.data)/params['batch_size'], iter_num_train)
                    iter_num_train += 1
                else:
                    writer.add_scalar(f'Loss/{phase}', loss.item()/params['batch_size'], iter_num_val)
                    writer.add_scalar(f'Accuracy/{phase}', torch.sum(preds == labels.data)/params['batch_size'], iter_num_val)
                    iter_num_val += 1                    
                

            epoch_loss = current_loss / dataset_sizes[phase]
            epoch_acc = current_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Make a copy of the model if the accuracy on the validation set has improved
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_since = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_since // 60, time_since % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # Now we'll load in the best model weights and return it
    model.load_state_dict(best_model_wts)
    return model

# Train model

In [ ]:
base_model = train_model(vgg, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)

/home/esa/miniconda3/envs/openimg/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
train epoch (0):   0%|          | 0/266 [00:00<?, ?it/s]

Epoch 0/19
----------
Iterating through data...


val epoch (0):   0%|          | 0/89 [00:00<?, ?it/s]

train Loss: 2.4633 Acc: 0.0537
Iterating through data...


train epoch (1):   0%|          | 0/266 [00:00<?, ?it/s]

val Loss: 2.2914 Acc: 0.0411

Epoch 1/19
----------
Iterating through data...


train epoch (1):  77%|███████▋  | 206/266 [01:10<00:20,  2.98it/s]

# Tensorboard 

In [31]:
from torch.utils.tensorboard import SummaryWriter


writer = SummaryWriter()

for n_iter in range(100):
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)

# Other

In [20]:
dataset_sizes = {'train': len(training_set.list_IDs), 'val': len(validation_set.list_IDs)}
dataset_sizes

{'train': 21036, 'val': 7012}

In [ ]:
l_val = []
for i, (x, y) in enumerate(validation_generator):
    l_val.append(y.numpy()[0])

In [ ]:
l_train = []
for x, y in training_generator:
    l_train.append(y.numpy()[0])

In [ ]:
len(l_val), len(l_train)

In [ ]:
y_val = np.bincount(l_val)
ii_val = np.nonzero(y_val)[0]

y_train = np.bincount(l_train)
ii_train = np.nonzero(y_train)[0]

counts = dataset.class_to_idx.copy()
i = 0
for key in counts:
    counts[key] = [y_train[i], y_val[i], df_counts[key]]
    i += 1
counts

# Target: 224 x 224

In [ ]:
generators = {'train' : training_generator, 'val' : validation_generator}

In [ ]:
generators['train']